In [1]:
import torch
torch.cuda.is_available()

True

Data.py

In [4]:
import collections
from random import shuffle
import os
from os import listdir
from os.path import join

import numpy as np
from PIL import Image  # Import PIL for image decoding


def is_jpeg(filename):
    #return any(filename.endswith(extension) for extension in [".jpg", ".jpeg"])
    return any(filename.endswith(extension) for extension in [".png"])


def get_subdirs(directory):
    subdirs = sorted([join(directory, name) for name in sorted(os.listdir(directory)) if os.path.isdir(os.path.join(directory, name))])
    return subdirs


flatten = lambda l: [item for sublist in l for item in sublist]


class ExternalInputIterator:
    def __init__(self, imageset_dir, batch_size, random_shuffle=False):
        self.images_dir = imageset_dir
        self.batch_size = batch_size

        # ... (Rest of the initialization code remains the same)
    
        # First, figure out what are the inputs and what are the targets in your directory structure:
        # Get a list of filenames for the target (frontal) images
        self.frontals = np.array([join(imageset_dir, frontal_file) for frontal_file in sorted(os.listdir(imageset_dir)) if is_jpeg(frontal_file)])
        print(len(self.frontals ))
        
        # Get a list of lists of filenames for the input (profile) images for each person
        profile_files = [[join(person_dir, profile_file) for profile_file in sorted(os.listdir(person_dir)) if is_jpeg(profile_file)] for person_dir in get_subdirs(imageset_dir)]
       
        print(len(profile_files))
        # Build a flat list of frontal indices, corresponding to the *flattened* profile_files
        # The reason we are doing it this way is that we need to keep track of the multiple inputs corresponding to each target
        frontal_ind = []
        for ind, profiles in enumerate(profile_files):
            frontal_ind += [ind]*len(profiles)
        self.frontal_indices = np.array(frontal_ind)
        
        # Now that we have built frontal_indices, we can flatten profile_files
        self.profiles = np.array(flatten(profile_files))

        # Shuffle the (input, target) pairs if necessary: in practice, it is profiles and frontal_indices that get shuffled
        if random_shuffle:
            ind = np.array(range(len(self.frontal_indices)))
            shuffle(ind)
            self.profiles = self.profiles[ind]
            self.frontal_indices = self.frontal_indices[ind]
            print(len(self.frontal_indices) )


    def __iter__(self):
        self.i = 0
        self.n = len(self.frontal_indices)
        return self

    def __next__(self):
        profiles = []
        frontals = []

        for _ in range(self.batch_size):
            profile_filename = self.profiles[self.i]
            frontal_filename = self.frontals[self.frontal_indices[self.i]]

            # Use PIL to decode JPEG images
            with Image.open(profile_filename) as profile_img:
                profiles.append(np.array(profile_img))
            with Image.open(frontal_filename) as frontal_img:
                frontals.append(np.array(frontal_img))

            self.i = (self.i + 1) % self.n
        return (profiles, frontals)


"""class ImagePipeline:
    def __init__(self, imageset_dir, image_size=128, random_shuffle=False, batch_size=64, num_threads=2, device_id=0):
        self.eii = ExternalInputIterator(imageset_dir, batch_size, random_shuffle)
        self.iterator = iter(self.eii)
        self.num_inputs = len(self.eii.frontal_indices)
        self.image_size = image_size

    def epoch_size(self, name=None):
        return self.num_inputs
    def __len__(self):
        # Return the number of samples in the dataset
        return self.num_inputs
    def __iter__(self):
        return self

    def __next__(self):
        (images, targets) = next(self.iterator)

        # Perform resizing and normalization using NumPy
        resized_images = np.array([np.array(Image.fromarray(img).resize((self.image_size, self.image_size))) for img in images])
        resized_targets = np.array([np.array(Image.fromarray(target).resize((self.image_size, self.image_size))) for target in targets])

        # Normalize using mean and standard deviation
        normalized_images = (resized_images - 128.0) / 128.0
        normalized_targets = (resized_targets - 128.0) / 128.0

        return (normalized_images, normalized_targets)"""

class ImagePipeline:
    def __init__(self, imageset_dir, image_size=128, random_shuffle=False, batch_size=64, num_threads=2, device_id=0):
        self.eii = ExternalInputIterator(imageset_dir, batch_size, random_shuffle)
        self.iterator = iter(self.eii)
        self.num_inputs = len(self.eii.frontal_indices)
        self.image_size = image_size

    def epoch_size(self, name=None):
        return self.num_inputs

    def __len__(self):
        return self.num_inputs

    def __iter__(self):
        return self

    def __next__(self):
        (images, targets) = next(self.iterator)

        # Perform resizing and normalization using NumPy
        resized_images = np.array([np.array(Image.fromarray(img).resize((self.image_size, self.image_size))) for img in images])
        resized_targets = np.array([np.array(Image.fromarray(target).resize((self.image_size, self.image_size))) for target in targets])

        # Normalize using mean and standard deviation
        normalized_images = (resized_images - 128.0) / 128.0
        normalized_targets = (resized_targets - 128.0) / 128.0

        return (normalized_images, normalized_targets)

    def __getitem__(self, index):  # Added __getitem__ method
        # Advance the iterator to the desired index
        for _ in range(index):
            next(self.iterator)

        # Return the next batch
        return next(self)


In [8]:
x = ExternalInputIterator(imageset_dir=r"C:\Users\zed\Dataset\cropped",batch_size=32)

In [6]:
import collections
from random import shuffle
import os
from os import listdir
from os.path import join

import numpy as np
from PIL import Image  # Import PIL for image decoding


def is_jpeg(filename):
    # You can customize this function to match your preferred file extension(s)
    return any(filename.endswith(extension) for extension in [".jpg", ".jpeg", ".png"])


"""def get_subdirs(directory):
    # Ensure both "frontal_file" and "profile_file" folders exist
    subdirs = [
        d
        for d in sorted(os.listdir(directory))
        if os.path.isdir(os.path.join(directory, d))
        and all(
            os.path.isdir(os.path.join(d, subfolder))
            for subfolder in ["frontal_file", "profile_file"]
        )
    ]
    return sorted(subdirs)"""
def get_subdirs(directory):
    subdirs = sorted([join(directory, name) for name in sorted(os.listdir(directory)) if os.path.isdir(os.path.join(directory, name))])
    return subdirs



flatten = lambda l: [item for sublist in l for item in sublist]


class ExternalInputIterator:
    def __init__(self, imageset_dir, batch_size, random_shuffle=False):
        self.imageset_dir = imageset_dir
        self.batch_size = batch_size

        # Get subdirectories (assuming "frontal_file" and "profile_file" folders exist)
        self.person_dirs = get_subdirs(imageset_dir)

        # Collect frontal image paths (adjust according to your data structure)
        self.frontals = [
            join(person_dir, frontal_file)
            for person_dir in self.person_dirs
            for frontal_file in os.listdir(person_dir)
            if is_jpeg(frontal_file)
        ]
        #print(self.frontals)

        # Collect profile image paths (adjust according to your data structure)
        self.profile_files = [
            [
                join(person_dir, profile_file)
                for profile_file in os.listdir(person_dir)
                if is_jpeg(profile_file)
            ]
            for person_dir in self.person_dirs
        ]
        #print(len(self.profile_files))

        # Create a flattened list of frontal indices
        self.frontal_indices = np.array(
            [i for i, profiles in enumerate(self.profile_files) for _ in profiles]
        )
        #print(len(self.frontal_indices ))

        # Flatten profile_files for easier access
        self.profiles = np.array(flatten(self.profile_files))
        #rint(len(self.profiles ))

        # Shuffle if necessary
        if random_shuffle:
            """ind = np.array(range(len(self.frontal_indices)))
            shuffle(ind)
            self.profiles = self.profiles[ind]
            self.frontal_indices = self.frontal_indices[ind]"""
            ind = np.array(range(len(self.frontal_indices)))
            shuffle(ind)
            self.profiles = self.profiles[ind.astype(int)]
            self.frontal_indices = self.frontal_indices[ind.astype(int)]


        self.i = 0
        self.n = len(self.frontal_indices)

    def __iter__(self):
        return self

    def __next__(self):
        profiles = []
        frontals = []

        for _ in range(self.batch_size):
            profile_filename = self.profiles[self.i]
            frontal_filename = self.frontals[self.frontal_indices[self.i]]

            with Image.open(profile_filename) as profile_img:
                profiles.append(np.array(profile_img))
            with Image.open(frontal_filename) as frontal_img:
                frontals.append(np.array(frontal_img))

            self.i = (self.i + 1) % self.n

        return (profiles, frontals)

class ImagePipeline:
    def __init__(self, imageset_dir, image_size=128, random_shuffle=False, batch_size=64, num_threads=2, device_id=0):
        self.eii = ExternalInputIterator(imageset_dir, batch_size, random_shuffle)
        self.iterator = iter(self.eii)
        self.num_inputs = len(self.eii.frontal_indices)
        self.image_size = image_size

    def epoch_size(self, name=None):
        return self.num_inputs

    def __len__(self):
        return self.num_inputs

    def __iter__(self):
        return self

    def __next__(self):
        (images, targets) = next(self.iterator)

        # Perform resizing and normalization using NumPy
        resized_images = np.array([np.array(Image.fromarray(img).resize((self.image_size, self.image_size))) for img in images])
        resized_targets = np.array([np.array(Image.fromarray(target).resize((self.image_size, self.image_size))) for target in targets])

        # Normalize using mean and standard deviation
        normalized_images = (resized_images - 128.0) / 128.0
        normalized_targets = (resized_targets - 128.0) / 128.0

        return (normalized_images, normalized_targets)

    def __getitem__(self, index):  # Added __getitem__ method
        # Advance the iterator to the desired index
        for _ in range(index):
            next(self.iterator)

        # Return the next batch
        return next(self)


Edited data.py

In [1]:
import collections
from random import shuffle
import os
from os import listdir
from os.path import join

import numpy as np
from PIL import Image  # Import PIL for image decoding


def is_jpeg(filename):
    # You can customize this function to match your preferred file extension(s)
    return any(filename.endswith(extension) for extension in [".jpg", ".jpeg", ".png"])


"""def get_subdirs(directory):
    # Ensure both "frontal_file" and "profile_file" folders exist
    subdirs = [
        d
        for d in sorted(os.listdir(directory))
        if os.path.isdir(os.path.join(directory, d))
        and all(
            os.path.isdir(os.path.join(d, subfolder))
            for subfolder in ["frontal_file", "profile_file"]
        )
    ]
    return sorted(subdirs)"""
def get_subdirs(directory):
    subdirs = sorted([join(directory, name) for name in sorted(os.listdir(directory)) if os.path.isdir(os.path.join(directory, name))])
    return subdirs



flatten = lambda l: [item for sublist in l for item in sublist]


class ExternalInputIterator:
    def __init__(self, imageset_dir, batch_size, random_shuffle=False):
        self.imageset_dir = imageset_dir
        self.batch_size = batch_size

        # Get subdirectories (assuming "frontal_file" and "profile_file" folders exist)
        self.person_dirs = get_subdirs(imageset_dir)

        # Collect frontal image paths (adjust according to your data structure)
        self.frontals = np.array(
            [join(imageset_dir, frontal_file) for frontal_file in sorted(os.listdir(imageset_dir)) if is_jpeg(frontal_file)]
        )
        print(len(self.frontals))

        # Collect profile image paths (adjust according to your data structure)
        self.profile_files = [
            [
                join(person_dir, profile_file)
                for profile_file in os.listdir(person_dir)
                if is_jpeg(profile_file)
            ]
            for person_dir in self.person_dirs
        ]
        #print(len(self.profile_files))

        # Create a flattened list of frontal indices
        self.frontal_indices = np.array(
            [i for i, profiles in enumerate(self.profile_files) for _ in profiles]
        )
        print(len(self.frontal_indices ))

        # Flatten profile_files for easier access
        self.profiles = np.array(flatten(self.profile_files))
        print(len(self.profiles ))

        # Shuffle if necessary
        if random_shuffle:
            """ind = np.array(range(len(self.frontal_indices)))
            shuffle(ind)
            self.profiles = self.profiles[ind]
            self.frontal_indices = self.frontal_indices[ind]"""
            ind = np.array(range(len(self.frontal_indices)))
            shuffle(ind)
            self.profiles = self.profiles[ind.astype(int)]
            self.frontal_indices = self.frontal_indices[ind.astype(int)]


        self.i = 0
        self.n = len(self.frontal_indices)

    def __iter__(self):
        return self

    def __next__(self):
        profiles = []
        frontals = []

        for _ in range(self.batch_size):
            profile_filename = self.profiles[self.i]
            #frontal_filename = self.frontals[self.frontal_indices[self.i]]
            frontal_filename = self.frontals[self.i]

            with Image.open(profile_filename) as profile_img:
                profiles.append(np.array(profile_img))
            with Image.open(frontal_filename) as frontal_img:
                frontals.append(np.array(frontal_img))

            self.i = (self.i + 1) % self.n

        return (profiles, frontals)

class ImagePipeline:
    def __init__(self, imageset_dir, image_size=128, random_shuffle=False, batch_size=64, num_threads=2, device_id=0):
        self.eii = ExternalInputIterator(imageset_dir, batch_size, random_shuffle)
        self.iterator = iter(self.eii)
        self.num_inputs = len(self.eii.frontal_indices)
        self.image_size = image_size

    def epoch_size(self, name=None):
        return self.num_inputs

    def __len__(self):
        return self.num_inputs

    def __iter__(self):
        return self

    def __next__(self):
        (images, targets) = next(self.iterator)

        # Perform resizing and normalization using NumPy
        resized_images = np.array([np.array(Image.fromarray(img).resize((self.image_size, self.image_size))) for img in images])
        resized_targets = np.array([np.array(Image.fromarray(target).resize((self.image_size, self.image_size))) for target in targets])

        # Normalize using mean and standard deviation
        normalized_images = (resized_images - 128.0) / 128.0
        normalized_targets = (resized_targets - 128.0) / 128.0

        return (normalized_images, normalized_targets)

    def __getitem__(self, index):  # Added __getitem__ method
        # Advance the iterator to the desired index
        for _ in range(index):
            next(self.iterator)

        # Return the next batch
        return next(self)


network.py

In [2]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
from torch.autograd import Variable


def weights_init(m):
    classname = m.__class__.__name__
    
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


''' Generator network for 128x128 RGB images '''
class G(nn.Module):
    
    def __init__(self):
        super(G, self).__init__()
        
        self.main = nn.Sequential(
            # Input HxW = 128x128
            nn.Conv2d(3, 16, 4, 2, 1), # Output HxW = 64x64
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.Conv2d(16, 32, 4, 2, 1), # Output HxW = 32x32
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.Conv2d(32, 64, 4, 2, 1), # Output HxW = 16x16
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(64, 128, 4, 2, 1), # Output HxW = 8x8
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Conv2d(128, 256, 4, 2, 1), # Output HxW = 4x4
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(256, 512, 4, 2, 1), # Output HxW = 2x2
            nn.MaxPool2d((2,2)),
            # At this point, we arrive at our low D representation vector, which is 512 dimensional.

            nn.ConvTranspose2d(512, 256, 4, 1, 0, bias = False), # Output HxW = 4x4
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False), # Output HxW = 8x8
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False), # Output HxW = 16x16
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias = False), # Output HxW = 32x32
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, 4, 2, 1, bias = False), # Output HxW = 64x64
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 3, 4, 2, 1, bias = False), # Output HxW = 128x128
            nn.Tanh()
        )

    
    def forward(self, input):
        output = self.main(input)
        return output


''' Discriminator network for 128x128 RGB images '''
class D(nn.Module):
    
    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
                                  nn.Conv2d(3, 16, 4, 2, 1),
                                  nn.LeakyReLU(0.2, inplace = True),
                                  nn.Conv2d(16, 32, 4, 2, 1),
                                  nn.BatchNorm2d(32),
                                  nn.LeakyReLU(0.2, inplace = True),
                                  nn.Conv2d(32, 64, 4, 2, 1),
                                  nn.BatchNorm2d(64),
                                  nn.LeakyReLU(0.2, inplace = True),
                                  nn.Conv2d(64, 128, 4, 2, 1),
                                  nn.BatchNorm2d(128),
                                  nn.LeakyReLU(0.2, inplace = True),
                                  nn.Conv2d(128, 256, 4, 2, 1),
                                  nn.BatchNorm2d(256),
                                  nn.LeakyReLU(0.2, inplace = True),
                                  nn.Conv2d(256, 512, 4, 2, 1),
                                  nn.BatchNorm2d(512),
                                  nn.LeakyReLU(0.2, inplace = True),
                                  nn.Conv2d(512, 1, 4, 2, 1, bias = False),
                                  nn.Sigmoid()
                                  )
    
    
    def forward(self, input):
        output = self.main(input)
        return output.view(-1)

main.py

In [3]:
from __future__ import print_function
import time
import math
import random
import os
from os import listdir
from os.path import join
from PIL import Image

import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torchvision.utils as vutils
from torch.autograd import Variable
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

#from nvidia.dali.plugin.pytorch import DALIGenericIterator

#from data import ImagePipeline
#import network

np.random.seed(42)
random.seed(10)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(999)
torch.cuda.manual_seed(999)
# Where is your training dataset at?
datapath = r"C:\Users\zed\Dataset\cropped\frontal_file"

# You can also choose which GPU you want your model to be trained on below:
gpu_id = 0
device = torch.device("cuda", gpu_id)

"""train_pipe = ImagePipeline(datapath, image_size=128, random_shuffle=True, batch_size=30, device_id=gpu_id)
train_pipe.build()
m_train = train_pipe.epoch_size()
print("Size of the training set: ", m_train)
train_pipe_loader = DALIGenericIterator(train_pipe, ["profiles", "frontals"], m_train)"""
# Assuming you have the modified ImagePipeline class from the previous responses
train_pipe = ImagePipeline(datapath, image_size=128, random_shuffle=True, batch_size=32, device_id=gpu_id)
# No need to call build() without DALI

# Use a standard PyTorch DataLoader instead of DALIGenericIterator
#train_pipe_loader = DataLoader(train_pipe, batch_size=train_pipe.batch_size)
m_train = train_pipe.epoch_size()
#train_pipe_loader = DataLoader(train_pipe, batch_size=32,)
train_pipe_loader = DataLoader(train_pipe,)
# Generator:
#netG = network.G().to(device)
#netG.apply(network.weights_init)
netG = G().to(device)
netG.apply(weights_init)

# Discriminator:
#netD = network.D().to(device)
#netD.apply(network.weights_init)
netD = D().to(device)
netD.apply(weights_init)

# Here is where you set how important each component of the loss function is:
L1_factor = 0
L2_factor = 1
GAN_factor = 0.0005

criterion = nn.BCELoss() # Binary cross entropy loss

# Optimizers for the generator and the discriminator (Adam is a fancier version of gradient descent with a few more bells and whistles that is used very often):
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999), eps = 1e-8)

# Create a directory for the output files
try:
    os.mkdir('output')
except OSError:
    pass

start_time = time.time()

# Let's train for 30 epochs (meaning, we go through the entire training set 30 times):
for epoch in range(3):
    
    # Lets keep track of the loss values for each epoch:
    loss_L1 = 0
    loss_L2 = 0
    loss_gan = 0
    
    # Your train_pipe_loader will load the images one batch at a time
    # The inner loop iterates over those batches:
    print("starting...")
    with tqdm(total=len(train_pipe_loader), desc=f"Epoch {epoch+1}") as pbar:
        for i, data in enumerate(train_pipe_loader, 0):
        
        # These are your images from the current batch:
        #profile = data[0]['profiles']
        #frontal = data[0]['frontals']
            profile = data[0].view(32, 3, 128, 128)
            frontal = data[1].view(32, 3, 128, 128)
        
        
        # TRAINING THE DISCRIMINATOR
            netD.zero_grad()
            real = Variable(frontal).type('torch.FloatTensor').to(device)
            target = Variable(torch.ones(real.size()[0])).to(device)
            output = netD(real)
        # D should accept the GT images
            errD_real = criterion(output, target)
        
            profile = Variable(profile).type('torch.FloatTensor').to(device)
            generated = netG(profile)
            target = Variable(torch.zeros(real.size()[0])).to(device)
            output = netD(generated.detach()) # detach() because we are not training G here

            pbar.update(1)
        
        # D should reject the synthetic images
            errD_fake = criterion(output, target)
        
            errD = errD_real + errD_fake
            errD.backward()
        # Update D
            optimizerD.step()
        
        # TRAINING THE GENERATOR
            netG.zero_grad()
            target = Variable(torch.ones(real.size()[0])).to(device)
            output = netD(generated)
        
        # G wants to :
        # (a) have the synthetic images be accepted by D (= look like frontal images of people)
            errG_GAN = criterion(output, target)
        
        # (b) have the synthetic images resemble the ground truth frontal image
            errG_L1 = torch.mean(torch.abs(real - generated))
            errG_L2 = torch.mean(torch.pow((real - generated), 2))
        
            errG = GAN_factor * errG_GAN + L1_factor * errG_L1 + L2_factor * errG_L2
        
            loss_L1 += errG_L1.item()
            loss_L2 += errG_L2.item()
            loss_gan += errG_GAN.item()
        
            errG.backward()
        # Update G
            optimizerG.step()
    
    if epoch == 0:
        print('First training epoch completed in ',(time.time() - start_time),' seconds')
    if epoch > 0:
        print(f"Epoch: {epoch} is starting..")
    # reset the DALI iterator
    #train_pipe_loader.reset()

    # Print the absolute values of three losses to screen:
    print('[%d/30] Training absolute losses: L1 %.7f ; L2 %.7f BCE %.7f' % ((epoch + 1), loss_L1/m_train, loss_L2/m_train, loss_gan/m_train,))

    # Save the inputs, outputs, and ground truth frontals to files:
    vutils.save_image(profile.data, 'output/%03d_input.jpg' % epoch, normalize=True)
    vutils.save_image(real.data, 'output/%03d_real.jpg' % epoch, normalize=True)
    vutils.save_image(generated.data, 'output/%03d_generated.jpg' % epoch, normalize=True)

    

    # Save the pre-trained Generator as well
    torch.save(netG,'output/netG_%d.pt' % epoch)

111
111
111
starting...


Epoch 1: 100%|███████████████████████████████████████████████████████████████████████| 111/111 [05:17<00:00,  2.86s/it]


First training epoch completed in  317.498330116272  seconds
[1/30] Training absolute losses: L1 0.3119904 ; L2 0.1406782 BCE 7.2579693
starting...


Epoch 2: 100%|███████████████████████████████████████████████████████████████████████| 111/111 [05:20<00:00,  2.89s/it]


Epoch: 1 is starting..
[2/30] Training absolute losses: L1 0.2082021 ; L2 0.0687924 BCE 5.7170552
starting...


Epoch 3: 100%|███████████████████████████████████████████████████████████████████████| 111/111 [05:16<00:00,  2.86s/it]


Epoch: 2 is starting..
[3/30] Training absolute losses: L1 0.1621829 ; L2 0.0447100 BCE 4.1597956


In [10]:
from __future__ import print_function
import time
import math
import random
import os
from os import listdir
from os.path import join
from PIL import Image

import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torchvision.utils as vutils
from torch.autograd import Variable
import torch
from torch.utils.data import DataLoader

#from nvidia.dali.plugin.pytorch import DALIGenericIterator

#from data import ImagePipeline
#import network

np.random.seed(42)
random.seed(10)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(999)

# Where is your training dataset at?
datapath = r"C:\Users\zed\Dataset\cropped"

# You can also choose which GPU you want your model to be trained on below:
gpu_id = 0
device = torch.device("cuda", gpu_id)

"""train_pipe = ImagePipeline(datapath, image_size=128, random_shuffle=True, batch_size=30, device_id=gpu_id)
train_pipe.build()
m_train = train_pipe.epoch_size()
print("Size of the training set: ", m_train)
train_pipe_loader = DALIGenericIterator(train_pipe, ["profiles", "frontals"], m_train)"""
# Assuming you have the modified ImagePipeline class from the previous responses
train_pipe = ImagePipeline(datapath, image_size=128, random_shuffle=True, batch_size=32,device_id=gpu_id)
# No need to call build() without DALI

# Use a standard PyTorch DataLoader instead of DALIGenericIterator
#train_pipe_loader = DataLoader(train_pipe, batch_size=train_pipe.batch_size)

train_pipe_loader = DataLoader(train_pipe)

m_train = train_pipe.epoch_size()
print("Size of the training set: ", m_train)
print(len(train_pipe_loader))


Size of the training set:  211
211


In [8]:
print(train_pipe_loader.__dict__)

{'dataset': <__main__.ImagePipeline object at 0x0000022A09AD0E90>, 'num_workers': 0, 'prefetch_factor': None, 'pin_memory': False, 'pin_memory_device': '', 'timeout': 0, 'worker_init_fn': None, '_DataLoader__multiprocessing_context': None, '_dataset_kind': 0, 'batch_size': 30, 'drop_last': False, 'sampler': <torch.utils.data.sampler.SequentialSampler object at 0x0000022A09AD0750>, 'batch_sampler': <torch.utils.data.sampler.BatchSampler object at 0x0000022A09AD12D0>, 'generator': None, 'collate_fn': <function default_collate at 0x0000022A06A75080>, 'persistent_workers': False, '_DataLoader__initialized': True, '_IterableDataset_len_called': None, '_iterator': None}


In [182]:
import torch
from torch.utils.data import DataLoader

# Assuming you have the modified ImagePipeline class and train_pipe_loader initialization...

# Check if train_pipe_loader is iterable
if not hasattr(train_pipe_loader, '__iter__'):
    raise TypeError("train_pipe_loader is not iterable. Double-check its setup.")

# Convert to a list for processing (optional, but safer for data handling)
data_list = list(train_pipe_loader)

# Check if data_list contains dictionaries
if not isinstance(data_list[0], dict):
    raise TypeError("First element of data_list is not a dictionary. Check data structure.")

# Extract and print keys
for data in data_list:
    print("Keys of current dictionary:", list(data.keys()))


TypeError: First element of data_list is not a dictionary. Check data structure.

In [12]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms

# Define image transformations (adjust as needed)
transform = transforms.Compose([
    transforms.Resize(128),  # Resize to image_size
    transforms.ToTensor(),   # Convert to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Optional normalization
])

# Load datasets using PyTorch's ImageFolder
profiles_dataset = datasets.ImageFolder(r"C:\Users\zed\Dataset\cropped\profile_file", transform=transform)
frontals_dataset = datasets.ImageFolder(r"C:\Users\zed\Dataset\cropped\frontal_file", transform=transform)
print(len(profiles_dataset))
print(len(frontals_dataset))
# Create dataloaders
profiles_loader = torch.utils.data.DataLoader(profiles_dataset, batch_size=batch_size, shuffle=True)
frontals_loader = torch.utils.data.DataLoader(frontals_dataset, batch_size=batch_size, shuffle=True)

# Function for displaying images without DALI
def show_images(image_batch):
    columns = 4
    rows = (image_batch.shape[0] + 1) // columns
    fig = plt.figure(figsize=(32, (32 // columns) * rows))
    gs = gridspec.GridSpec(rows, columns)

    for j in range(image_batch.shape[0]):
        plt.subplot(gs[j])
        plt.axis("off")
        plt.imshow(image_batch[j].permute(1, 2, 0))  # Permute for correct display

# Get image batches
profiles_batch = next(iter(profiles_loader))[0]
frontals_batch = next(iter(frontals_loader))[0]

# Display images
show_images(profiles_batch)
show_images(frontals_batch)


FileNotFoundError: Couldn't find any class folder in C:\Users\zed\Dataset\cropped\profile_file.

In [11]:
for i, data in enumerate(train_pipe_loader, 0):
    x = data[0]
    if len(x) > 1:
        break
plt.imshow(x)

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have access to 'train_pipe_loader'

# Choose an index to access a specific image (replace with your desired index)
image_index = 0

# Extract the image (adapt based on your data structure)
image = None
for i, data in enumerate(train_pipe_loader, 0):
    if i == image_index:
        image = data  # Assuming data contains the image
        break

# Convert to NumPy array if needed
if isinstance(image, torch.Tensor):
    image = image.cpu().numpy()

# Process the image for display (optional)
# ... (e.g., normalize, convert channels)

# Display the image
plt.imshow(image)
plt.show()
